In [2]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
PATH = "C:/Users/jackl/OneDrive/Documents/finance_research/japan_qe/"

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [3]:
# jp_df = pd.read_csv(PATH+'raw_data/comp_global/all_japanese_stocks.csv', parse_dates=['datadate'])
# jp_df = jp_df.dropna(subset=['prccd'])
# jp_df['imp_market_val'] = jp_df['cshoc'] * jp_df['prccd'] / 10e5
# jp_df['sedol'] = jp_df['sedol'].astype(str)
# jp_df['prccd'] = jp_df['prccd'].astype('float64')
# jp_df['imp_market_val'] = jp_df['imp_market_val'].astype('float64')

In [4]:
const_df_full = pd.read_csv(PATH+'raw_data/datastream/topix_nikkei_constituents_v2.csv', parse_dates=['date_'])
const_df_full = const_df_full.dropna(subset=['close_', 'weight'])
const_df_full = const_df_full[const_df_full['close_'] != 0] 
const_df_full['weight'] = const_df_full['weight'] / 100
const_df_full['close_'] = const_df_full['close_'].astype('float64')
const_df_full['mktval'] = const_df_full['mktval'].astype('float64')
const_df_full['month_yr_index'] = const_df_full['date_'].apply(lambda x: (x.year, x.month))
const_df_full = const_df_full[const_df_full['date_'] >= pd.to_datetime('2010-11-30')]

In [5]:
nikkei_df_reo = pd.read_pickle(PATH+'checkpoint_data/nikkei_weights_bloomberg.pkl')
nikkei_df_reo['SEDOL'] = nikkei_df_reo['SEDOL'].astype(str)
nikkei_df_reo = nikkei_df_reo.dropna(subset=['weight'])
nikkei_df_reo['indexlistmnem'] = ['LJAPDOWA' for i in range(len(nikkei_df_reo))]
nikkei_df_reo['weight'] = nikkei_df_reo['weight'] / 100
nikkei_df_reo['month_yr_index'] = nikkei_df_reo['date'].apply(lambda x: (x.year, x.month))

merge_df = pd.read_pickle(PATH+'checkpoint_data/sedol_constintcode_match.pkl')
merge_df['sedol'] = merge_df['sedol'].astype(str)
nikkei_df_reo = pd.merge_asof(nikkei_df_reo.sort_values('date'), 
                              merge_df.sort_values('merged_date'), 
                              left_by=['SEDOL'], 
                              right_by=['sedol'],
                              left_on=['date'],
                              right_on=['merged_date'],
                              direction='nearest').drop(columns=['merged_date', 'SEDOL'])

nikkei_df_reo_pre_aug = nikkei_df_reo[nikkei_df_reo['date'] <= pd.to_datetime('2011-08-01')]
nikkei_df_reo_pre_aug = nikkei_df_reo_pre_aug[['date', 'weight', 'constintcode', 'indexlistmnem', 'month_yr_index']]
nikkei_df_reo_pre_aug.columns = ['date_', 'weight', 'constintcode', 'indexlistmnem', 'month_yr_index']
nikkei_df_reo_pre_aug = nikkei_df_reo_pre_aug.dropna(subset=['constintcode'])

In [6]:
comn_df = pd.read_csv(PATH+'raw_data/comp_global/company_names.csv')
comn_df = comn_df.drop_duplicates(['sedol'])[['sedol', 'conm']]
comn_df['sedol'] = comn_df['sedol'].astype(str)

# Merging

In [7]:
const_df = const_df_full[['indexlistmnem', 'constintcode', 'date_', 'weight', 'month_yr_index']]

In [8]:
control_df = const_df[(const_df['indexlistmnem']=='LJAPDOWA') & 
                      (const_df['month_yr_index']==(2011, 8))]

nikkei_df_reo_pre_aug_added_list = []
for index, df in nikkei_df_reo_pre_aug.groupby(['month_yr_index']): 
    add_df = control_df[~control_df['constintcode'].isin(df['constintcode'])]
    add_df['month_yr_index'] = [index for i in range(len(add_df))]
    add_df['date_'] = [df.iloc[0]['date_'] for i in range(len(add_df))]
    df_added = pd.concat([df, add_df])
    nikkei_df_reo_pre_aug_added_list.append(df_added)

nikkei_df_reo_pre_aug_added = pd.concat(nikkei_df_reo_pre_aug_added_list)

<ipython-input-8-0cd56638b9cd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_df['month_yr_index'] = [index for i in range(len(add_df))]
<ipython-input-8-0cd56638b9cd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_df['date_'] = [df.iloc[0]['date_'] for i in range(len(add_df))]


In [9]:
nk_df = const_df[const_df['indexlistmnem']=='LJAPDOWA']
tp_df = const_df[const_df['indexlistmnem']=='LTOKYOSE']

missing_months_1 = [(2020, 10), (2020, 11), (2020, 12), (2021, 1), (2021, 2)]
missing_months_2 = [(2015, 11), (2015, 12)]
missing_months_3 = [(2014, 11), (2014, 12), (2015, 1), (2015, 2), (2015, 3), (2015, 4), (2015, 5)]
missing_months_4 = [(2015, 7)]
missing_months_5 = [(2016, 2)]
missing_months_list = [missing_months_1, missing_months_2, missing_months_3, missing_months_4, missing_months_5]

gen_mon_dfs_list = []
for missing_month_x in missing_months_list: 
    start_index = missing_month_x[0]
    before_index = (start_index[0], start_index[1]-1)
    gen_mon_df = nk_df[nk_df['month_yr_index']==before_index]
    
    counter = 1
    for missing_month in missing_month_x:
        gen_mon_df_copy = gen_mon_df.copy()
        gen_mon_df_copy['date_'] = tp_df[tp_df['month_yr_index']==missing_month]['date_'].iloc[0]
        gen_mon_df_copy['month_yr_index'] = [missing_month for i in range(len(gen_mon_df))]
        gen_mon_dfs_list.append(gen_mon_df_copy)
        counter += 1
        
gen_mon_dfs_full = pd.concat(gen_mon_dfs_list)

In [10]:
const_df_fixed = pd.concat([nikkei_df_reo_pre_aug_added, gen_mon_dfs_full, nk_df, tp_df])
const_df_fixed['constintcode'] = const_df_fixed['constintcode'].astype('int64')

In [11]:
#get SEDOLs
merge_df = pd.read_pickle(PATH+'checkpoint_data/sedol_constintcode_match.pkl')
const_df_final = pd.merge_asof(const_df_fixed.sort_values('date_'), 
                               merge_df.sort_values('merged_date'), 
                               by=['constintcode'], 
                               left_on=['date_'],
                               right_on=['merged_date'],
                               direction='nearest').dropna(subset=['sedol']).drop_duplicates(['indexlistmnem', 'date_', 'sedol'])

In [12]:
# print_full(const_df_final.groupby(['date_', 'indexlistmnem']).sum())

In [15]:
# #get free float factor
# fffactor_match_df = const_df_full[const_df_full['indexlistmnem']=='LTOKYOSE'][['constintcode', 'date_', 'fffactor', 'month_yr_index']]
# fffactor_match_df = fffactor_match_df[fffactor_match_df['fffactor'] != 0]
# const_df_final_1 = pd.merge(const_df_final, 
#                             fffactor_match_df.drop(columns=['date_']), 
#                             on=['month_yr_index', 'constintcode'],
#                             how='left')

const_df_final_1 = const_df_final

In [16]:
const_df_final_1['month'] = const_df_final_1['date_'].dt.month
const_df_final_1['yr'] = const_df_final_1['date_'].dt.year

In [17]:
const_df_final_1['sedol'] = const_df_final_1['sedol'].astype(str)

In [18]:
const_df_final_1 = pd.merge(const_df_final_1, 
                            comn_df, 
                            on='sedol', 
                            how='left')

In [20]:
const_df_final_1.to_pickle(PATH+'checkpoint_data/index_weights.pkl')

In [15]:
# print_full(const_df_final.groupby(['date_', 'indexlistmnem']).count())